## Introduction

The Transmission Entry Capacity (TEC) Register is a record of generation projects that hold contracts for Transmission Entry Capacity (TEC) with National Grid ESO. This includes both connected projects and future connection projects, as well as projects that are directly connected to the National Electricity Transmission System (NETS) or connected at distribution level (and which have a Bilateral Embedded Generator Agreement (BEGA)). TEC is the maximum capacity in MW that a generator is permitted to export into the NETS. It is one of the two types of connection capacity included in transmission offers. The other is Connection Entry Capacity (CEC), which is the maximum potential output of a generation asset onto the NETS. CEC is often set higher than TEC to allow for changes over years in outputs without needing to adapt or modify infrastructure.

In [511]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import requests
import os
import sys
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

Get package versions

In [564]:
import plotly, matplotlib, bs4
import streamlit as st

for package in [pd, np, st, plotly, matplotlib, bs4]:
    print(f"{package.__name__}=={package.__version__}")

pandas==2.2.2
numpy==1.26.4
streamlit==1.32.0
plotly==5.22.0
matplotlib==3.8.4
bs4==4.12.3


## Download data

Download TEC data directly from [NESO dataportal](https://www.neso.energy/data-portal/transmission-entry-capacity-tec-register)

In [513]:
def download_data(url, filename):
    resp = requests.get(url)
    os.makedirs("datastore", exist_ok=True)
    if resp.ok:

        with open(filename, "wb") as f:
            f.write(resp.content)
    else:
        return

In [514]:
url = "https://api.neso.energy/dataset/cbd45e54-e6e2-4a38-99f1-8de6fd96d7c1/resource/17becbab-e3e8-473f-b303-3806f43a6a10/download/tec-register-27-09-2024.csv"
filename = "tecregister.csv"
filepath = f"./datastore/{filename}"

download_data(url, filepath)


data = pd.read_csv(filepath)
data.head()

,Project Name,Customer Name,Connection Site,Stage,MW Connected,MW Increase / Decrease,Cumulative Total Capacity (MW),MW Effective From,Project Status,Agreement Type,HOST TO,Plant Type,Project ID,Project Number
0,[23-0225] UPPER GREENFIELDS - 175MW BESS,Lower 48 Energy BESS Limited,New Deer 2 400kV Substation,NaN,0.0,175.00,175.00,2033-10-31,Scoping,Direct Connection,SHET,Energy Storage System,a0l8e000000ewjHAAQ,PRO-003682
1,012 NEP Coventry West,NEW ENERGY PARTNERSHIP LIMITED,Berkswell GSP,NaN,0.0,92.40,92.40,2034-10-31,Scoping,Embedded,NGET,Energy Storage System,a0l8e000000f3zXAAQ,PRO-003804
2,17 Acres and BESS,AEUK BATTERY PROJECT II LTD,Tealing 275/33kV Substation,NaN,0.0,95.76,95.76,2032-06-30,Scoping,Direct Connection,SHET,Energy Storage System,a0l8e0000012ezRAAQ,PRO-004392
3,Abedare,UK POWER RESERVE LIMITED,Upperboat 132kV Substation,NaN,10.0,0.00,10.00,NaN,Built,Embedded,NGET,CCGT (Combined Cycle Gas Turbine),a0l4L0000005iWRQAY,PRO-000001
4,Aberarder Extension Wind Farm,Renewable Energy Systems Limited,Tomatin 275/132 kV Substation,NaN,0.0,138.00,138.00,2031-04-30,Scoping,Direct Connection,SHET,Energy Storage System;Wind Onshore,a0l8e0000010wQ0AAI,PRO-002589


### Scrape last updated from data portal

In [515]:
def extract_last_date_updated(url, selector):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }

    try:
        # Fetch the webpage content
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.text, "lxml")

        # Extract the text from the target element using the provided selector
        last_updated = soup.select(selector)
        extracted_texts = [element.text.strip() for element in last_updated if element.name == "td" and "ago" in element.text]

        return extracted_texts[0] if extracted_texts else "Element not found"

    except requests.RequestException as e:
        return [f"Error fetching the URL: {e}"]


url = "https://www.neso.energy/data-portal/transmission-entry-capacity-tec-register"
selector = "div.layout-region-resources .views-field-changed.is-active"

extracted_text = extract_last_date_updated(url, selector)
print(extracted_text)

1 day ago


In [516]:
# Convert to datetime and format date
data["MW Effective From"] = pd.to_datetime(data["MW Effective From"], errors="coerce")


def change_names():
    data.rename(
        columns={
            "MW Increase / Decrease": "MW Change",
            "Cumulative Total Capacity (MW)": "Connection Cap (MW)",
            "MW Effective From": "Connection Date",
        },
        inplace=True,
    )
    return data


data = change_names()
data.head()

,Project Name,Customer Name,Connection Site,Stage,MW Connected,MW Change,Connection Cap (MW),Connection Date,Project Status,Agreement Type,HOST TO,Plant Type,Project ID,Project Number
0,[23-0225] UPPER GREENFIELDS - 175MW BESS,Lower 48 Energy BESS Limited,New Deer 2 400kV Substation,NaN,0.0,175.00,175.00,2033-10-31,Scoping,Direct Connection,SHET,Energy Storage System,a0l8e000000ewjHAAQ,PRO-003682
1,012 NEP Coventry West,NEW ENERGY PARTNERSHIP LIMITED,Berkswell GSP,NaN,0.0,92.40,92.40,2034-10-31,Scoping,Embedded,NGET,Energy Storage System,a0l8e000000f3zXAAQ,PRO-003804
2,17 Acres and BESS,AEUK BATTERY PROJECT II LTD,Tealing 275/33kV Substation,NaN,0.0,95.76,95.76,2032-06-30,Scoping,Direct Connection,SHET,Energy Storage System,a0l8e0000012ezRAAQ,PRO-004392
3,Abedare,UK POWER RESERVE LIMITED,Upperboat 132kV Substation,NaN,10.0,0.00,10.00,NaT,Built,Embedded,NGET,CCGT (Combined Cycle Gas Turbine),a0l4L0000005iWRQAY,PRO-000001
4,Aberarder Extension Wind Farm,Renewable Energy Systems Limited,Tomatin 275/132 kV Substation,NaN,0.0,138.00,138.00,2031-04-30,Scoping,Direct Connection,SHET,Energy Storage System;Wind Onshore,a0l8e0000010wQ0AAI,PRO-002589


In [517]:
data.describe()

,Stage,MW Connected,MW Change,Connection Cap (MW),Connection Date
count,251.000000,1896.000000,1896.000000,1896.000000,1595
mean,1.705179,39.042468,277.981428,340.112596,2030-11-18 04:19:06.583072256
min,1.000000,0.000000,-1120.000000,0.000000,2019-09-06 00:00:00
25%,1.000000,0.000000,47.500000,57.000000,2027-07-07 00:00:00
50%,2.000000,0.000000,150.000000,200.000000,2030-10-30 00:00:00
75%,2.000000,0.000000,400.000000,437.000000,2033-10-31 00:00:00
max,6.000000,2363.000000,4800.000000,4800.000000,2039-10-31 00:00:00
std,0.800458,189.214526,403.084425,453.060015,NaN


In [518]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1896 entries, 0 to 1895
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Project Name         1896 non-null   object        
 1   Customer Name        1896 non-null   object        
 2   Connection Site      1896 non-null   object        
 3   Stage                251 non-null    float64       
 4   MW Connected         1896 non-null   float64       
 5   MW Change            1896 non-null   float64       
 6   Connection Cap (MW)  1896 non-null   float64       
 7   Connection Date      1595 non-null   datetime64[ns]
 8   Project Status       1896 non-null   object        
 9   Agreement Type       1896 non-null   object        
 10  HOST TO              1896 non-null   object        
 11  Plant Type           1896 non-null   object        
 12  Project ID           1896 non-null   object        
 13  Project Number       1896 non-nul

Missing Values

Tracking Missing Values

In [519]:
data.isna().sum()

Project Name              0
Customer Name             0
Connection Site           0
Stage                  1645
MW Connected              0
MW Change                 0
Connection Cap (MW)       0
Connection Date         301
Project Status            0
Agreement Type            0
HOST TO                   0
Plant Type                0
Project ID                0
Project Number            0
dtype: int64

Data shows 301 missing values for `Connection Date` (Date), which signifies that 301 customers are either already connected or have not completed their contractual requirements etc.

In [520]:
data.groupby(["HOST TO"])["Connection Cap (MW)"].sum()

HOST TO
NGET    486966.335
OFTO      4338.400
SHET     80322.587
SPT      73226.160
Name: Connection Cap (MW), dtype: float64

> **Analysis**

The grouped analysis indicates a clear disparity in total capacity across the different hosts, with NGET being the primary player in the UK electricity transmission landscape. Understanding these dynamics is crucial for stakeholders in energy policy, investment, and operational planning moving forward.

 - **Dominance of NGET**: NGET accounts for the vast majority of the cumulative total capacity, with approximately 88.5% of the total capacity across all hosts. This indicates that the National Grid plays a critical role in electricity transmission and likely serves as the primary backbone for the electricity network in the UK.
 - **Relative Contribution of OFTO**: The Offshore Transmission Operators have a significantly smaller capacity compared to NGET, contributing only about 0.9% of the total. This may suggest a growing but still limited role in the overall electricity transmission capacity, likely due to the current scale of offshore projects.
 - **Capacity Distribution in Scotland**: Both SHET and SPT, which are specific to the Scottish region, show moderate capacities at 80.3GW and 73.2GW, respectively. Together, they account for approximately 12% of the total capacity. This reflects Scotland's significant investment in renewable energy and its distinct grid requirements.


 With the increasing emphasis on renewable energy, especially offshore wind, the role of OFTOs may grow.

In [521]:
data.groupby(["HOST TO"]).agg({"Project Status": "count", "Connection Cap (MW)": "sum"})

,Project Status,Connection Cap (MW)
HOST TO,,
NGET,1147,486966.335
OFTO,17,4338.400
SHET,350,80322.587
SPT,382,73226.160


In [522]:
capacity_by_TO_plant = data.pivot_table(
    index="Plant Type",
    columns="HOST TO",
    values="Connection Cap (MW)",
    fill_value=0,
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
capacity_by_TO_plant

HOST TO,NGET,OFTO,SHET,SPT,Total
Plant Type,,,,,
Biomass,3083.000,0.0,0.000,55.00,3138.000
CCGT (Combined Cycle Gas Turbine),34822.000,0.0,2090.000,20.00,36932.000
CCGT (Combined Cycle Gas Turbine);Demand,2406.000,0.0,0.000,0.00,2406.000
CCGT (Combined Cycle Gas Turbine);Demand;Energy Storage System,21600.000,0.0,0.000,0.00,21600.000
CCGT (Combined Cycle Gas Turbine);Energy Storage System,5714.000,0.0,0.000,0.00,5714.000
CCGT (Combined Cycle Gas Turbine);Energy Storage System;OCGT (Open Cycle Gas Turbine),7600.000,0.0,0.000,0.00,7600.000
CCGT (Combined Cycle Gas Turbine);Reactive Compensation,1517.000,0.0,0.000,0.00,1517.000
CHP (Combined Heat and Power),734.040,0.0,0.000,120.00,854.040
CHP (Combined Heat and Power);Energy Storage System,1268.000,0.0,0.000,0.00,1268.000


> **Analysis**

Analysis of Energy Generation Capacity by Plant Type
This analysis examines the connection capacity (in MW) of various energy plant types across different host connections, summarized from the provided pivot table. The data offers insights into the distribution of energy generation capabilities among different technologies.

**Overview of the Data**
Total Connection Capacity: The overall capacity across all plant types is approximately 644,853.48 MW (or 644.85 GW), indicating a significant generation capacity within the network.

Key Host Connections: The data, as analysed previously, is divided into four primary host connections:
 + NGET (National Grid Electricity Transmission)
 + OFTO (Offshore Transmission Owner)
 + SHET (Scottish Hydro Electric Transmission)
 + SPT (Scottish Power Transmission)

 <br><br>
<u>Key Findings</u>


 - __Dominance of CCGT and Energy Storage Systems__: CCGT (Combined Cycle Gas Turbine) plants, including various configurations, account for a total of approximately 36.93GW connected primarily to NGET.
Energy Storage Systems dominate with an impressive total of 134.54GW, showcasing their critical role in stabilizing the grid and integrating renewable energy sources.
 - __Wind Offshore Leading Renewable Generation__: Wind Offshore plants show a substantial capacity of 127.28GW, indicating their pivotal role in the current energy mix. They are well-represented across all connections, especially NGET and SHET. Wind Onshore contributes a capacity of 28.95 GW, reinforcing the importance of both offshore and onshore wind in the renewable sector.
 - __Limited Capacity from Conventional Sources__: Conventional sources like Coal, Oil & AGT, and Gas Reciprocating show relatively low contributions (e.g., 2.05GW, 61.5 MW, and 661.78 MW respectively). This trend reflects a broader shift towards cleaner energy sources. Nuclear Power provides a significant capacity of 19.09 GW, still an essential component of the low-carbon energy landscape.

<br></br>
<u>Diversity in Energy Storage Solutions:</u>

**Energy Storage Systems** not only enhance grid reliability but also support various generation types, indicated by their multiple configurations (e.g., Energy Storage with PV, Wind).
The substantial combined capacity of Energy Storage Systems signifies their growing importance in enabling the transition to renewable energy.

 - High Contributions from PV Arrays: PV Array (Photo Voltaic/Solar) plants account for 7.40 GW, with various configurations also contributing to overall capacity, underscoring solar power's expanding role.
 - Reactive Compensation: The data includes various configurations for reactive compensation, which, while individually small (e.g., 21 MW), cumulatively support grid stability.


**Summary**
 - The analysis of connection capacities reveals a clear transition towards renewable energy sources, particularly wind and energy storage technologies. The dominance of CCGT and Energy Storage Systems illustrates the critical roles they play in ensuring a reliable and flexible energy grid. As the energy landscape evolves, these insights could guide future investments and policies aimed at optimizing energy production and reducing carbon emissions.

In [523]:
unpivot_capacity_by_TO_plant = (
    capacity_by_TO_plant.drop("Total", axis=1)
    .reset_index()
    .melt(id_vars="Plant Type", value_name="Connection Cap (MW)", var_name="HOST TO")
)
unpivot_capacity_by_TO_plant

,Plant Type,HOST TO,Connection Cap (MW)
0,Biomass,NGET,3083.00
1,CCGT (Combined Cycle Gas Turbine),NGET,34822.00
2,CCGT (Combined Cycle Gas Turbine);Demand,NGET,2406.00
3,CCGT (Combined Cycle Gas Turbine);Demand;Energ...,NGET,21600.00
4,CCGT (Combined Cycle Gas Turbine);Energy Stora...,NGET,5714.00
...,...,...,...
183,Tidal,SPT,0.00
184,Waste,SPT,0.00
185,Wind Offshore,SPT,11606.00
186,Wind Onshore,SPT,13603.12


In [524]:
fig = px.bar(
    data_frame=unpivot_capacity_by_TO_plant,
    x="Plant Type",
    y="Connection Cap (MW)",
    color="HOST TO",
    color_continuous_scale=px.colors.diverging.RdBu_r,
    height=900,
)
fig.show()

In [525]:
data.columns

Index(['Project Name', 'Customer Name', 'Connection Site', 'Stage',
       'MW Connected', 'MW Change', 'Connection Cap (MW)', 'Connection Date',
       'Project Status', 'Agreement Type', 'HOST TO', 'Plant Type',
       'Project ID', 'Project Number'],
      dtype='object')

In [526]:
data.shape

(1896, 14)

In [527]:
colors = ["#1000ee", "#2B5D18", "#FFD700", "#2CFFe5"]
sun = px.sunburst(
    data_frame=data,
    values="Connection Cap (MW)",
    color="HOST TO",
    color_discrete_sequence=colors * (data.shape[0] // 4 + 1),
    path=["HOST TO", "Plant Type", "Project Status"],
    hover_data=["Project Name", "Connection Cap (MW)"],
    width=800,
    height=800,
    title="Connection Capacity by Host TO, Plant Type, and Project Status. (click to Expand)",
)

sun.show()

In [528]:
status_proj_cap = data.groupby(["Project Status", "HOST TO"]).agg(
    {"Connection Cap (MW)": "sum", "Project Name": "count"}
)
status_proj_cap

Connection Cap (MW)  Project Name
Project Status                   HOST TO                                   
Awaiting Consents                NGET               53442.460           120
                                 SHET                1633.800            11
                                 SPT                 3597.440            26
Built                            NGET               58544.670           152
                                 OFTO                4338.400            17
                                 SHET                6643.850            80
                                 SPT                 7050.800            63
Consents Approved                NGET               29085.065            81
                                 SHET                5723.490            39
                                 SPT                 3533.630            28
Scoping                          NGET              335186.880           772
                                 SHET               66321.447           220
                                 SPT                58332.790           262
Under Construction/Commissioning NGET               10707.260            22
                                 SPT                  711.500             3

> Analysis of Project Status by Connection Capacity (in GW)

The dataset categorizes projects by their connection capacity and status, summarized as follows:

1. **Scoping**:
   - **Capacity**: 459.84 GW
   - **Number of Projects**: 1,254
   - **Insight**: This category has the largest capacity, indicating a substantial number of projects in the planning phase. This suggests significant future potential for energy generation.

2. **Built**:
   - **Capacity**: 76.58 GW
   - **Number of Projects**: 312
   - **Insight**: Operational projects contribute a notable portion to the energy supply, reflecting established capacity.

3. **Awaiting Consents**:
   - **Capacity**: 58.67 GW
   - **Number of Projects**: 157
   - **Insight**: This significant capacity shows many projects waiting for approvals, which may delay future capacity additions to the grid.

4. **Consents Approved**:
   - **Capacity**: 38.34 GW
   - **Number of Projects**: 148
   - **Insight**: These projects are positioned for development but have less capacity than built and scoping projects.

5. **Under Construction/Commissioning**:
   - **Capacity**: 11.42 GW
   - **Number of Projects**: 25
   - **Insight**: This is the lowest capacity category, indicating limited current construction activity compared to planning stages.

### Summary

The analysis reveals a significant pipeline of projects in the scoping phase, suggesting future growth in capacity. Operational projects, while substantial, are outpaced by planned capacity, indicating potential challenges in meeting demand if regulatory approvals lag. The distribution highlights the importance of advancing projects through the consent and construction phases to ensure timely energy generation.

In [529]:
to_status_group = data.groupby("HOST TO")["Project Status"].count()
to_status_group

HOST TO
NGET    1147
OFTO      17
SHET     350
SPT      382
Name: Project Status, dtype: int64

Connection Capacity distribution by Project Status and by HOST TO

In [530]:

def create_hover_text(row, primary_col, secondary_col):
    hover_text = f"<b>{row[primary_col]}</b><br>"
    hover_text += f"<b>Total:</b> {row['Total']:.2f} MW<br>"
    for status in sorted(data[secondary_col].unique()):
        if status in row.index and row[status] > 0:  # Only show non-zero values
            hover_text += f"<b>{status}:</b> {row[status]:.2f} MW<br>"
    return hover_text

def create_pie_data(primary_col, secondary_col):
    # Prepare the data for the pie chart
    proj_cap = data.groupby(primary_col)['Connection Cap (MW)'].sum().reset_index()
    proj_cap = proj_cap.rename(columns={'Connection Cap (MW)': 'Total'})

    # Create a pivot table for secondary data
    secondary_data = data.pivot_table(values='Connection Cap (MW)',
                                      index=primary_col,
                                      columns=secondary_col,
                                      aggfunc='sum',
                                      fill_value=0)

    # Merge the pivot table with proj_cap
    proj_cap = pd.merge(proj_cap, secondary_data, left_on=primary_col, right_index=True)

    # Create hover text
    proj_cap['hover_text'] = proj_cap.apply(lambda row: create_hover_text(row, primary_col, secondary_col), axis=1)

    return proj_cap

# Create data for both pie charts
status_data = create_pie_data('Project Status', 'HOST TO')
host_to_data = create_pie_data('HOST TO', 'Project Status')

# Create subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=('Connection Capacity by Project Status', 'Connection Capacity by HOST TO'))

# Add traces for both pie charts
fig.add_trace(go.Pie(labels=status_data['Project Status'], 
                     values=status_data['Total'], 
                     name='Project Status',
                     hole=0.45,
                     hoverinfo='text',
                     text=status_data['hover_text'],
                     textinfo='percent+label',
                     hovertemplate='%{text}<extra></extra>'), 1, 1)

fig.add_trace(go.Pie(labels=host_to_data['HOST TO'], 
                     values=host_to_data['Total'], 
                     name='HOST TO',
                     hole=0.45,
                     hoverinfo='text',
                     text=host_to_data['hover_text'],
                     textinfo='percent+label',
                     hovertemplate='%{text}<extra></extra>'), 1, 2)

# Update layout
fig.update_layout(
    title_text='Connection Capacity Distribution',
    width=1200,
    height=700,
    legend=dict(orientation='h', yanchor='bottom', y=-0.1, xanchor='center', x=0.5),
)

# Update trace settings
fig.update_traces(textposition='auto', textfont_size=10)

# Show the plot
fig.show()

## Capacities Timeline

In [531]:
data.columns

Index(['Project Name', 'Customer Name', 'Connection Site', 'Stage',
       'MW Connected', 'MW Change', 'Connection Cap (MW)', 'Connection Date',
       'Project Status', 'Agreement Type', 'HOST TO', 'Plant Type',
       'Project ID', 'Project Number'],
      dtype='object')

In [532]:

# Grouping the data by 'Connection Date'
time_group = data.groupby(['Connection Date', 'HOST TO']).agg({
    'Connection Cap (MW)': 'sum',
    'Project Name': 'count',
    'Plant Type': 'nunique',
    'MW Change': 'sum'
}).reset_index()

# Create a scatter plot
timeline_plot = px.scatter(
    data_frame=time_group,
    x='Connection Date',
    y='Connection Cap (MW)',
    size='Project Name',  
    hover_name='Connection Date', 
    color='HOST TO',
    hover_data={
        'Project Name': True,
        'Plant Type': True,
        'MW Change': True
    },
    title='Timeline of Connection Capacity, Projects, Plant Types, and MW Change',
    labels={
        'Connection Cap (MW)': 'Total Entry Capacity (MW)',
        'MW Change': 'Total MW Change',
        'Project Name': 'Number of Projects',
        'Plant Type': 'Unique Plant Type'
    },
    # template='plotly_white'
)

# Update layout for improved aesthetics
timeline_plot.update_layout(
    yaxis_title='Total Connection Capacity (MW)',
    xaxis_title='Connection Date',
    legend_title='HOST TO',
    showlegend=True
)

# Show the plot
timeline_plot.show()


In [563]:
time_group.to_csv('time_group.csv')